In [14]:
import pandas as pd
import numpy as np
import glob

In [117]:
from treasury_ml_utils.snowflake_utils import SnowflakeConnector

snowflake_client = SnowflakeConnector("joshua.choy@transferwise.com")

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://transferwise.okta-emea.com/app/snowflake/exk4istpb5gZUyV8u0i7/sso/saml?SAMLRequest=nZJbb%2BIwEIX%2FSuR9TpxkQwELqNgCKuqNhYBWfTPJQC0cO%2FU4BP59HS6r7kP7sG%2BRc2a%2BM3Omd3sopLcHg0KrPomCkHigMp0Lte2TZTrxO8RDy1XOpVbQJ0dAcjvoIS9kyYaVfVNzeK8ArecaKWTNjz6pjGKao0CmeAHIbMYWw6dHFgch44hgrMORS0mOwrHerC0ZpXVdB%2FXPQJstjcMwpGGXOlUj%2BUE%2BIcrvGaXRVmdaXksObqYvEBENkwbhFI4wuxT%2BEuq8gu8o67MI2X2azvzZyyIl3vA63Z1WWBVgFmD2IoPl%2FPFsAJ2DeRq3k7gTQOVnoKzh0o8CVLreSL6DTBdlZV3jwH3RDeRU6q1w65qO%2BqTciTxZ2%2FF4eT95T%2BvuH%2FE8Xf1uJYtFcngxD4t9tF5DupHJ08O4lQ4z4q2u4cZNuFPECqaqidS6pzC%2B8cPIj7tp2GVhwlrtoHXTeSXeyEUqFLenyqtv51ThBkwtEAK9s9yHAvjJJi9L%2BncCCoddItCW69b2dXlcdapQtCmipk125Hw%2B7GTFDP5jKT36ucHlGJ9dPtPRTEuRHb2JNgW3X8cXBdHpReT%2B5iRlUHAhh3luANHFKKWu7wxw627emgoIHZyp

In [120]:
pnl_df = snowflake_client.fetch(

'''
select HOUR_TIMESTAMP, ccy, sum(FX_PNL) as total_fx_pnl
from RPT_TREASURY.TREASURY_FX_PNL_HOURLY
where ccy in ('MYR', 'PHP', 'INR', 'IDR', 'KRW', 'THB', 'VND')
and HOUR_TIMESTAMP::date >= '2024-01-01'
group by all

'''
)

exposure_df = snowflake_client.fetch(
'''
with position as
(SELECT ccy
    , case
          when minute(created_at) > 30
              then time_slice(created_at, 1, 'hour', 'end')
          else time_slice(created_at, 1, 'hour', 'start') end
             as hour_timestamp
    , sum(amount) as hour_position
FROM ARM.OPEN_POSITION_AUDIT
WHERE TYPE IN ('CUSTOMER',
              'BORDERLESS',
              'HEDGE')
 AND hour_timestamp >= '2024-01-01'
and ccy in ('THB','MYR', 'PHP', 'INR', 'IDR', 'KRW', 'THB', 'VND')
group by all),

rates as (
    select MINUTE_TIMESTAMP, target_ccy, SOURCE_CCY, value
    from RATE_HISTORICAL.ANALYTICS_RATES
    where TARGET_CCY in ('GBP', 'USD')
    and minute_timestamp >= '2024-01-01')

select position.*,
       rates_usd.value as rate_to_usd,
       rates_gbp.value as rate_to_gbp
from position
left join rates as rates_usd
    on rates_usd.MINUTE_TIMESTAMP = hour_timestamp
    and rates_usd.TARGET_CCY = 'USD'
    and rates_usd.SOURCE_CCY = position.ccy
left join rates as rates_gbp
    on rates_gbp.MINUTE_TIMESTAMP = hour_timestamp
    and rates_gbp.TARGET_CCY = 'GBP'
    and rates_gbp.SOURCE_CCY = position.ccy
'''

)

In [123]:
pnl_df

,hour_timestamp,ccy,total_fx_pnl
0,2025-12-06 20:00:00,KRW,1.792595
1,2025-12-06 16:00:00,MYR,3.321754
2,2025-12-06 13:00:00,THB,0.353833
3,2025-12-06 13:00:00,MYR,16.336907
4,2025-12-06 00:00:00,PHP,23.831337
...,...,...,...
127563,2026-01-25 14:00:00,KRW,-2.871368
127564,2026-01-12 23:00:00,KRW,451.691308
127565,2026-01-13 00:00:00,MYR,676.675411
127566,2026-01-13 00:00:00,IDR,5442.472830


In [125]:
#pnl_df = pd.read_csv('/Users/joshua.choy/Downloads/total_fx_pnl_2024_onwards.csv')
# exposure_df = pd.read_csv('/Users/joshua.choy/Downloads/exposure_2024_onwards.csv')

pnl_df['hour_timestamp'] = pd.to_datetime(pnl_df['hour_timestamp'])
exposure_df['hour_timestamp'] = pd.to_datetime(exposure_df['hour_timestamp'])

In [126]:
print(exposure_df.columns)
print(pnl_df.columns)

Index(['ccy', 'hour_timestamp', 'hour_position', 'rate_to_usd', 'rate_to_gbp'], dtype='object')
Index(['hour_timestamp', 'ccy', 'total_fx_pnl'], dtype='object')


In [127]:
sorted_pnl = pnl_df.sort_values(by=['hour_timestamp'])
sorted_exposure = exposure_df.sort_values(by=['hour_timestamp'])
df = pd.merge(exposure_df, pnl_df[['hour_timestamp', 'ccy', 'total_fx_pnl']], on=["hour_timestamp", 'ccy'], how='left')
df['USD_RATE'] = 1/df['rate_to_usd']
df['GBP_RATE'] = 1/df['rate_to_gbp']

In [128]:
df

,ccy,hour_timestamp,hour_position,rate_to_usd,rate_to_gbp,total_fx_pnl,USD_RATE,GBP_RATE
0,INR,2024-02-13 04:00:00,6.673321e+07,0.012047,0.009553,-112.790256,83.005462,104.682446
1,VND,2024-02-13 03:00:00,1.388228e+11,0.000041,0.000032,-18.106381,24449.877751,30769.230769
2,IDR,2024-02-13 02:00:00,-1.633046e+10,0.000064,0.000051,-1692.059389,15600.624025,19685.039370
3,MYR,2024-02-13 00:00:00,5.826352e+07,0.209864,0.166209,3539.460427,4.764991,6.016521
4,VND,2024-02-13 07:00:00,1.314341e+11,0.000041,0.000032,-26.143879,24449.877751,30864.197531
...,...,...,...,...,...,...,...,...
127570,KRW,2025-09-12 21:00:00,3.018011e+09,0.000718,0.000529,-8.989915,1393.145723,1888.930865
127571,IDR,2025-09-12 21:00:00,2.531125e+11,0.000061,0.000045,-117.565499,16420.361248,22222.222222
127572,MYR,2025-09-12 18:00:00,1.274182e+08,0.237812,0.175358,-13055.407086,4.205002,5.702620
127573,VND,2025-09-12 17:00:00,1.388271e+11,0.000038,0.000028,33.184633,26385.224274,35842.293907


In [129]:
monday_df = df[df['hour_timestamp'].dt.dayofweek == 0]
monday_df.head()

,ccy,hour_timestamp,hour_position,rate_to_usd,rate_to_gbp,total_fx_pnl,USD_RATE,GBP_RATE
40,INR,2024-02-12 23:00:00,-4.810952e+08,0.012049,0.009543,-248.995303,82.995128,104.793243
41,PHP,2024-02-12 23:00:00,4.207310e+06,0.017852,0.014138,-54.774980,56.017388,70.730362
42,VND,2024-02-12 23:00:00,1.444439e+11,0.000041,0.000032,15.908074,24449.877751,30864.197531
51,MYR,2024-02-12 23:00:00,5.838246e+07,0.209820,0.166174,2025.659477,4.765990,6.017789
65,IDR,2024-02-12 23:00:00,-4.839243e+09,0.000064,0.000051,-737.736390,15600.624025,19685.039370


In [130]:
monday_df.loc[monday_df['hour_timestamp'].dt.hour == 8]

,ccy,hour_timestamp,hour_position,rate_to_usd,rate_to_gbp,total_fx_pnl,USD_RATE,GBP_RATE
495,INR,2024-06-10 08:00:00,-3.744239e+06,0.011975,0.009413,-1004.805097,83.504518,106.238314
550,MYR,2024-08-12 08:00:00,2.569584e+07,0.224706,0.175985,-4466.615894,4.450259,5.682302
751,PHP,2024-06-10 08:00:00,5.919955e+08,0.017007,0.013367,10272.527218,58.800009,74.808304
1046,VND,2024-04-15 08:00:00,3.670676e+10,0.000040,0.000032,-3658.762792,25125.628141,31347.962382
1047,MYR,2024-04-15 08:00:00,4.175280e+07,0.209238,0.167732,-4405.617859,4.779247,5.961892
...,...,...,...,...,...,...,...,...
127182,MYR,2025-08-25 08:00:00,-4.043892e+06,0.238010,0.176258,291.213762,4.201504,5.673501
127392,KRW,2025-08-25 08:00:00,1.771976e+09,0.000722,0.000534,282.859225,1385.617293,1871.257485
127393,PHP,2025-08-25 08:00:00,1.848217e+09,0.017629,0.013055,-30.456352,56.723428,76.596657
127510,VND,2025-08-25 08:00:00,1.913670e+10,0.000038,0.000028,78.336015,26315.789474,35587.188612


In [84]:
df

,CCY,HOUR_TIMESTAMP,HOUR_POSITION,RATE_TO_USD,RATE_TO_GBP,TOTAL_FX_PNL,USD_RATE,GBP_RATE
0,INR,2024-01-05 22:00:00,4.160243e+09,0.012028,0.009457,19.190648,83.137959,105.742897
1,VND,2024-01-03 12:00:00,-1.098420e+10,0.000041,0.000032,-28.332121,24390.243902,30769.230769
2,PHP,2024-01-03 10:00:00,8.031918e+08,0.017994,0.014250,-316.733494,55.575007,70.177408
3,VND,2024-01-05 21:00:00,-1.297157e+10,0.000041,0.000032,71.009220,24390.243902,31055.900621
4,PHP,2024-01-05 22:00:00,7.315278e+08,0.017994,0.014147,-14.060273,55.575007,70.685865
...,...,...,...,...,...,...,...,...
127556,THB,2026-01-19 06:00:00,1.554233e+08,0.031995,0.023886,-16.119476,31.254981,41.866054
127557,IDR,2026-01-19 05:00:00,-2.776481e+10,0.000059,0.000044,37.270619,16920.473773,22675.736961
127558,INR,2026-01-19 02:00:00,5.592257e+06,0.011030,0.008236,-74.074592,90.665119,121.419638
127559,IDR,2026-01-19 01:00:00,-9.741423e+10,0.000059,0.000044,-51.384839,16891.891892,22624.434389


In [132]:
def filter_df(df, day_of_week:int, ccy:str, hour_of_day:int):
    mask = (
        (df['hour_timestamp'].dt.dayofweek == day_of_week) & 
        (df['hour_timestamp'].dt.hour == hour_of_day) & 
        (df['ccy'] == ccy )
    )

    return df.loc[mask]


In [131]:
df.loc[df['ccy'] == 'PHP']

,ccy,hour_timestamp,hour_position,rate_to_usd,rate_to_gbp,total_fx_pnl,USD_RATE,GBP_RATE
13,PHP,2024-02-13 00:00:00,-2.968222e+07,0.017858,0.014143,-608.691818,55.997626,70.705357
22,PHP,2024-02-11 10:00:00,3.530139e+08,0.017874,0.014153,0.198429,55.946873,70.655399
23,PHP,2024-02-11 16:00:00,2.800666e+08,0.017874,0.014153,-20.117863,55.946873,70.655399
28,PHP,2024-02-10 15:00:00,5.811534e+08,0.017874,0.014153,1.163421,55.946873,70.655399
29,PHP,2024-02-08 06:00:00,7.368757e+07,0.017892,0.014166,-7.158172,55.889964,70.592056
...,...,...,...,...,...,...,...,...
127535,PHP,2025-10-05 11:00:00,9.161336e+08,0.017245,0.012796,30.222229,57.988159,78.147590
127543,PHP,2025-10-05 03:00:00,1.057949e+09,0.017245,0.012796,-1.242386,57.988159,78.147590
127547,PHP,2025-10-27 22:00:00,-6.524021e+08,0.016996,0.012743,3086.946883,58.836335,78.476304
127548,PHP,2025-10-27 21:00:00,-5.603344e+08,0.016980,0.012733,-6180.442710,58.893856,78.534854


In [133]:
ccys = ['INR', 'VND', 'PHP', 'IDR', 'MYR', 'THB', 'KRW']    

In [134]:
print(filter_df(df, 0, 'INR', 0)['hour_position'].mean())
print(filter_df(df, 0, 'INR', 0)['total_fx_pnl'].mean())

print(filter_df(df, 0, 'THB', 0)['hour_position'].mean())
print(filter_df(df, 0, 'THB', 0)['total_fx_pnl'].mean())

print(filter_df(df, 0, 'PHP', 0)['hour_position'].mean())
print(filter_df(df, 0, 'PHP', 0)['total_fx_pnl'].mean())

-308878643.8253211
-854.0314350656331
-122843921.04018342
-550.8354530867136
46200720.36587157
1043.7008347268968
